In [ ]:
import experiments.runners as runners

from tabpfn import TabPFNClassifier
from sklearn.ensemble import RandomForestClassifier
from data import load_us_perm_visas, load_credit_default

rf_model = RandomForestClassifier()
pfn_model = TabPFNClassifier()

In [ ]:
# if you don't have GPU and cuda, change task_type
# note: catboost requires python 3.12, so need to set up a new evironment (ok, since tabpfn requires only python > 3.9)
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(iterations=1000, 
                           task_type="GPU",
                           devices='0:1')#, class_weights=class_weights)

In [ ]:
new_run = runners.run(rf_model, load_us_perm_visas)

In [ ]:
new_run.pipe

In [ ]:
import warnings
# Ignore FutureWarning, of which TabPFNClassifier has a lot!
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
new_run.fit()

In [ ]:
new_run.predict()

In [ ]:
new_run.score()

# Testing new data loaders

In [1]:
from data.loaders import load_credit_default

df = load_credit_default(as_frame=True)

In [2]:
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
